<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Mixtral_8x7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U datasets transformers accelerate peft trl bitsandbytes sentencepiece interpret
!pip install colab-env --quiet

!pip install -U bitsandbytes -q

In [2]:
!nvidia-smi

Mon Nov 25 04:57:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Choose the Mixtral variant you want to use
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"  # Example: Instruction-tuned version

# BitsAndBytesConfig int-4 config (for reduced memory usage)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Set the desired GPU device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# or device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load the model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # device_map="auto", # Remove or comment out device_map="auto"
    trust_remote_code=True,
    quantization_config=bnb_config,
).to(device)  # Explicitly move the model to the device

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Revert padding strategy to default
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token  # Ensure default padding token is used
tokenizer.pad_token_id = tokenizer.eos_token_id  # Ensure default padding token ID is used

In [7]:
# Define your prompt
prompt = "Once upon a time, in a land far away, there lived a brave knight named Sir Reginald..."

# Prepare the input for the model
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text
outputs = base_model.generate(**inputs, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id)  # Generate up to 100 new tokens
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)

Once upon a time, in a land far away, there lived a brave knight named Sir Reginald...

Sir Reginald was a knight of great renown, known far and wide for his bravery and his skill with a sword. He was a true hero, and he had a heart of gold.

One day, Sir Reginald received a message from the king. The message was simple: there was a dragon terrorizing the kingdom, and the king needed Sir Reginald's help to slay it.

Sir Reginald didn't hesitate for a moment. He gathered his armor and his weapons, and he set out on his trusty steed to find the dragon.

It wasn't long before Sir Reginald found the dragon. It was a huge, fearsome creature, with scales as hard as iron and teeth as sharp as swords.

But Sir Reginald was not afraid. He charged at the dragon, his sword held high. The dragon roared and breathed fire, but Sir Reginald was not deterred. He fought with all his might, and eventually, he was able to strike the dragon down.

The kingdom was saved, and Sir Reginald was hailed as a he